# UMLS Entity Linking

In [1]:
import datasets
from pathlib import Path
from nen_util import get_annotation_dataframe
import pandas as pd

**Note:** For this notebook, we only use a small subset of GGPONC 2.0 for demonstration purposes
Change the version if you want to use any other GGPONC release

In [2]:
from xmen import load_config, load_kb

conf = load_config('xmen_ggponc3.yaml')
version = conf.data_version
version

'v2.0_agreement'

## Preparation

`xmen dict xmen_ggponc3.yaml`

`xmen index xmen_ggponc3.yaml --all --overwrite`

In [3]:
!mkdir -p output/$version
#!unzip -n -q data/$version/plain_text/plain_text.zip -d output/$version/plain_text

## Read Sentences

In [4]:
from ggponc import read_sentences
sent_df = read_sentences(version, 'data')
sent_df.head()

  0%|          | 0/14 [00:00<?, ?it/s]

,file,sentence_id,sentence
0,01_magenkarzinom_0192,0,Vitamine und essentielle Spurenelemente sind u...
1,01_magenkarzinom_0192,1,Mehr noch als bei Gesunden sollte bei Tumorpat...
2,01_magenkarzinom_0192,2,Am ehesten werden bei Patienten mit Malignomen...
3,01_magenkarzinom_0192,3,Bei oraler und enteraler Ernährung gelten für ...
4,01_magenkarzinom_0192,4,Die Nährstoffgehalte von bilanzierten Diäten s...


## Resolve Elliptical Compounds

In [5]:
from ggponc import ECCNPResolver
resolver = ECCNPResolver(**conf.eccnp)

/home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [6]:
resolver.transform('Chemo- und Strahlentherapie, BRAF-/MEK-Inhibitor')

'Chemotherapie und Strahlentherapie, BRAF-Inhibitor/MEK-Inhibitor,'

In [7]:
sent_df = resolver.set_df(sent_df)

  0%|          | 0/374 [00:00<?, ?it/s]

In [8]:
sent_df.to_parquet(f'output/{version}/sentences_resolved.parquet')

## Run NER Model

In [9]:
sent_df = pd.read_parquet(f'output/{version}/sentences_resolved.parquet')

In [10]:
# Drop excessively long pre-processed sentences, most like generation errors
sent_df['ratio'] = (sent_df.sentence_preprocessed.str.len() / sent_df.sentence.str.len()).sort_values()
drop_index = sent_df.ratio > 2
print('Resetting', drop_index.sum(), '/', len(drop_index), 'docs due to likely generation errors')
sent_df.loc[sent_df.ratio > 2, 'sentence_preprocessed'] = sent_df.sentence

Resetting 0 / 374 docs due to likely generation errors


In [11]:
from ggponc import NERTagger
ner = NERTagger()

In [12]:
d = ner.transform('Chemo- und Strahlentherapie, BRAF-/MEK-Inhibitor')
d.spans

{'entities': [Chemo-, Strahlentherapie, BRAF-/MEK-Inhibitor]}

In [13]:
d = ner.transform('Chemotherapie und Strahlentherapie, BRAF-Inhibitor/MEK-Inhibitor')
d.spans

{'entities': [Chemotherapie, Strahlentherapie, BRAF-Inhibitor, MEK-Inhibitor]}

In [14]:
ner_df = ner.set_df(sent_df)

  0%|          | 0/374 [00:00<?, ?it/s]

In [15]:
from ggponc import merge_sentence_docs
docs = merge_sentence_docs(ner_df.spacy_ner, ner_df.file)

In [16]:
from spacy import displacy

displacy.render(docs[0], style='span', options = {'spans_key' : 'entities'})

In [17]:
from xmen.data import from_spacy
ds = from_spacy(docs, span_key='entities', doc_id_key='file_name')

In [18]:
fname = f'output/{version}/ggponc_ner_spacy'
ds.save_to_disk(fname)
fname

Saving the dataset (0/1 shards):   0%|          | 0/14 [00:00<?, ? examples/s]

'output/v2.0_agreement/ggponc_ner_spacy'

# Run Entity Linker

In [19]:
ds = datasets.load_from_disk(f'output/{version}/ggponc_ner_spacy')

In [20]:
len([e for d in ds for e in d['entities']])

1177

In [21]:
from ggponc import EntityLinker
linker = EntityLinker(**conf.linker.ranking, candidate_generation_kwargs=conf.linker.candidate_generation)

/home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.3.1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.3.1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_down

[05/23/24 14:40:55] INFO     Loading hierarchical faiss index                                ]8;id=17394;file:///home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/xmen/linkers/sap_bert_linker.py\sap_bert_linker.py]8;;\:]8;id=66801;file:///home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/xmen/linkers/sap_bert_linker.py#153\153]8;;\

                    INFO     Loading index from                                                 ]8;id=334356;file:///home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/xmen/linkers/faiss_indexer.py\faiss_indexer.py]8;;\:]8;id=583298;file:///home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/xmen/linkers/faiss_indexer.py#64\64]8;;\
                             /home/Florian.Borchert/.cache/xmen/ggponc3/index/sapbert/embed_fai                    
                             ss_hier.pickle                                                                        

[05/23/24 14:40:59] INFO     Loaded index of type <class 'faiss.swigfaiss.IndexHNSWFlat'> and   ]8;id=941213;file:///home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/xmen/linkers/faiss_indexer.py\faiss_indexer.py]8;;\:]8;id=913061;file:///home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/xmen/linkers/faiss_indexer.py#66\66]8;;\
                             size 2465719                                                                          

/home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [22]:
result = linker.transform(ds)

[05/23/24 14:41:02] INFO     Expanding Abbreviations                                                  ]8;id=250871;file:///mnt/nfs/home/Florian.Borchert/workspace/ggponc/ggponc.py\ggponc.py]8;;\:]8;id=607450;file:///mnt/nfs/home/Florian.Borchert/workspace/ggponc/ggponc.py#136\136]8;;\

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

/home/Florian.Borchert/miniconda3/envs/ggponc/lib/python3.10/site-packages/scispacy/abbreviation.py:248: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)


[05/23/24 14:41:03] INFO     Generating Candidates                                                    ]8;id=117885;file:///mnt/nfs/home/Florian.Borchert/workspace/ggponc/ggponc.py\ggponc.py]8;;\:]8;id=864502;file:///mnt/nfs/home/Florian.Borchert/workspace/ggponc/ggponc.py#140\140]8;;\

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

...

  0%|          | 0/1177 [00:00<?, ?it/s]

  0%|          | 0/1177 [00:00<?, ?it/s]

Batches:   0%|          | 0/1177 [00:00<?, ?it/s]

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

In [23]:
result.save_to_disk(f'output/{version}/ggponc_xmen')

Saving the dataset (0/1 shards):   0%|          | 0/14 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14 [00:00<?, ? examples/s]

# Final Format

In [24]:
import datasets
result = datasets.load_from_disk(f'output/{version}/ggponc_xmen')

In [25]:
types = ['Clinical_Drug', 'Diagnostic', 'Therapeutic']

import random
show_indices = list(range(0, len(result['candidates'])))
random.shuffle(show_indices)
show_indices[0:10]

[13, 2, 3, 4, 11, 0, 5, 12, 7, 8]

In [26]:
get_annotation_dataframe(result['candidates'].select(show_indices[0:10]), linker.kb, 3, 0.0).query('type in @types').iloc[0:10]

  0%|          | 0/10 [00:00<?, ?it/s]

,document,text,type,start,end,cui,tuis,canonical,linker,confidence
72,27_supportive-therapie_0765,Strahlentherapie,Therapeutic,1162,1178,C1522449,[T061],Strahlentherapie,"[ngram, sapbert]",1.000000
73,27_supportive-therapie_0765,Strahlentherapie,Therapeutic,1162,1178,C0031765,[T061],Phototherapie,"[ngram, sapbert]",0.885240
74,27_supportive-therapie_0765,Strahlentherapie,Therapeutic,1162,1178,C0034619,[T170],radiotherapy,[sapbert],0.863900
75,27_supportive-therapie_0765,Steroidtherapie,Therapeutic,1313,1328,C0149783,[T061],Steroidtherapie,"[ngram, sapbert]",1.000000
76,27_supportive-therapie_0765,Steroidtherapie,Therapeutic,1313,1328,C3897353,[T061],Topical Steroid Therapy,"[ngram, sapbert]",0.813714
77,27_supportive-therapie_0765,Steroidtherapie,Therapeutic,1313,1328,C5667589,[T061],Pulse Steroids,"[ngram, sapbert]",0.806739
81,27_supportive-therapie_0765,Steroidtherapie,Therapeutic,1403,1418,C0149783,[T061],Steroidtherapie,"[ngram, sapbert]",1.000000
82,27_supportive-therapie_0765,Steroidtherapie,Therapeutic,1403,1418,C3897353,[T061],Topical Steroid Therapy,"[ngram, sapbert]",0.813714
83,27_supportive-therapie_0765,Steroidtherapie,Therapeutic,1403,1418,C5667589,[T061],Pulse Steroids,"[ngram, sapbert]",0.806739
90,27_supportive-therapie_0765,Bestrahlung,Therapeutic,1516,1527,C0851346,[T070],Strahlung,"[ngram, sapbert]",0.894799


In [27]:
get_annotation_dataframe(result['reranked'].select(show_indices[0:10]), linker.kb, 3, 0.0).query('type in @types').iloc[0:10]

  0%|          | 0/10 [00:00<?, ?it/s]

,document,text,type,start,end,cui,tuis,canonical,linker,confidence
72,27_supportive-therapie_0765,Strahlentherapie,Therapeutic,1162,1178,C1522449,[T061],Strahlentherapie,"[ngram, sapbert]",0.138755
73,27_supportive-therapie_0765,Strahlentherapie,Therapeutic,1162,1178,C0043308,[T061],Röntgenstrahlentherapie,[sapbert],0.063443
74,27_supportive-therapie_0765,Strahlentherapie,Therapeutic,1162,1178,C0948312,[T061],Radiotherapy to nose,[ngram],0.063037
75,27_supportive-therapie_0765,Steroidtherapie,Therapeutic,1313,1328,C0149783,[T061],Steroidtherapie,"[ngram, sapbert]",0.168302
76,27_supportive-therapie_0765,Steroidtherapie,Therapeutic,1313,1328,C0279025,[T061],Hormontherapie,"[ngram, sapbert]",0.063342
77,27_supportive-therapie_0765,Steroidtherapie,Therapeutic,1313,1328,C0279494,[T061],Oestrogentherapie,[sapbert],0.062963
81,27_supportive-therapie_0765,Steroidtherapie,Therapeutic,1403,1418,C0149783,[T061],Steroidtherapie,"[ngram, sapbert]",0.163678
82,27_supportive-therapie_0765,Steroidtherapie,Therapeutic,1403,1418,C0279025,[T061],Hormontherapie,"[ngram, sapbert]",0.063427
83,27_supportive-therapie_0765,Steroidtherapie,Therapeutic,1403,1418,C0279494,[T061],Oestrogentherapie,[sapbert],0.061039
90,27_supportive-therapie_0765,Bestrahlung,Therapeutic,1516,1527,C1522449,[T061],Strahlentherapie,[sapbert],0.116514


In [28]:
df = get_annotation_dataframe(result['reranked'], linker.kb, 1, 0.0)

  0%|          | 0/14 [00:00<?, ?it/s]

In [29]:
df.to_csv(f'output/{version}/entities_with_cuis.tsv', sep='\t', index=False)

# Analysis

In [31]:
df_analysis = get_annotation_dataframe(result['reranked'], linker.kb, 3, 0.0, show_progress=False)

In [37]:
german_umls = load_kb(Path(conf.cache_dir) / 'german_umls' / 'german_umls.jsonl')

In [42]:
def get_example(i):
    doc = df_analysis.loc[i].document
    d = result['reranked'].filter(lambda d: d['document_id'] == doc)[0]
    print(d['passages'])

### Language Mismatch

In [43]:
df_analysis[df_analysis.cui.map(lambda c: not c in german_umls.cui_to_entity)].sort_values('confidence', ascending=False)[0:20]

,document,text,type,start,end,cui,tuis,canonical,linker,confidence
2247,18_hodgkin-lymphom_0051,Therapie des Hodgkin-Lymphoms,Therapeutic,10077,10106,C5557287,[T170],HL Project Terminology,"[ngram, sapbert]",0.269659
2205,18_hodgkin-lymphom_0051,Therapie des Hodgkin Lymphoms,Therapeutic,9207,9236,C5557287,[T170],HL Project Terminology,"[ngram, sapbert]",0.264336
2248,18_hodgkin-lymphom_0051,Therapie des Hodgkin-Lymphoms,Therapeutic,10077,10106,C1880049,[T061],Regimen Used to Treat Hodgkin Lymphoma,[sapbert],0.246635
2206,18_hodgkin-lymphom_0051,Therapie des Hodgkin Lymphoms,Therapeutic,9207,9236,C1880049,[T061],Regimen Used to Treat Hodgkin Lymphoma,[sapbert],0.246276
2207,18_hodgkin-lymphom_0051,Therapie des Hodgkin Lymphoms,Therapeutic,9207,9236,C5670459,[T170],HL Radiation Therapy Table,[ngram],0.245979
2249,18_hodgkin-lymphom_0051,Therapie des Hodgkin-Lymphoms,Therapeutic,10077,10106,C5670464,[T170],HL Non-protocol Therapy Table,[ngram],0.242845
2499,21_hodentumoren_0260,schlechten Prognose,Other_Finding,1947,1966,C0278252,[T033],Prognosis bad,"[ngram, sapbert]",0.165607
2925,22_prostatakarzinom_0153,Padeliporfin,Clinical_Drug,57,69,C2698706,"[T116, T121, T123]",padeliporfin,"[ngram, sapbert]",0.158451
1155,07_malignes-melanom_0197,Mycophenolatmofetil,Clinical_Drug,2833,2852,C0209368,"[T109, T121]",Mycophenolate mofetil (substance),"[ngram, sapbert]",0.157939
2976,22_prostatakarzinom_0153,Padeliporfin,Clinical_Drug,637,649,C2698706,"[T116, T121, T123]",padeliporfin,"[ngram, sapbert]",0.157192


## Complex Entities

In [44]:
df_analysis[df_analysis.text.map(lambda s: len(s.split(' ')) > 5) & (df_analysis.type == 'Clinical_Drug')].sort_values('confidence', ascending=False)[0:20]

,document,text,type,start,end,cui,tuis,canonical,linker,confidence
1560,08_aktinische-keratose-und-plattenepithelkarzi...,"Diclofenac-Natrium 3% in Hyaluronsäure 2,5% Gel",Clinical_Drug,1865,1912,C2348252,[T200],Diclofenac Sodium Gel,"[ngram, sapbert]",0.095686
1590,08_aktinische-keratose-und-plattenepithelkarzi...,"Diclofenac-Natrium 3% in Hyaluronsäure 2,5% Gel",Clinical_Drug,2649,2696,C2348252,[T200],Diclofenac Sodium Gel,"[ngram, sapbert]",0.083908
1611,08_aktinische-keratose-und-plattenepithelkarzi...,"Diclofenac-Natrium 3% in Hyaluronsäure 2,5% Gel",Clinical_Drug,3184,3231,C2348252,[T200],Diclofenac Sodium Gel,"[ngram, sapbert]",0.083426
1548,08_aktinische-keratose-und-plattenepithelkarzi...,"Diclofenac-Natrium 3% in Hyaluronsäure 2,5% Gel",Clinical_Drug,1198,1245,C2348252,[T200],Diclofenac Sodium Gel,"[ngram, sapbert]",0.075504
1549,08_aktinische-keratose-und-plattenepithelkarzi...,"Diclofenac-Natrium 3% in Hyaluronsäure 2,5% Gel",Clinical_Drug,1198,1245,C4708783,[T200],Diclofenac diethylammonium 11.6 mg/g cutaneous...,"[ngram, sapbert]",0.062959
1550,08_aktinische-keratose-und-plattenepithelkarzi...,"Diclofenac-Natrium 3% in Hyaluronsäure 2,5% Gel",Clinical_Drug,1198,1245,C1164912,[T200],Product containing precisely diclofenac sodium...,"[ngram, sapbert]",0.062892
1591,08_aktinische-keratose-und-plattenepithelkarzi...,"Diclofenac-Natrium 3% in Hyaluronsäure 2,5% Gel",Clinical_Drug,2649,2696,C1164912,[T200],Product containing precisely diclofenac sodium...,"[ngram, sapbert]",0.062565
1592,08_aktinische-keratose-und-plattenepithelkarzi...,"Diclofenac-Natrium 3% in Hyaluronsäure 2,5% Gel",Clinical_Drug,2649,2696,C2348218,[T200],Sodium Hyaluronate Topical Hydrogel,"[ngram, sapbert]",0.062364
1612,08_aktinische-keratose-und-plattenepithelkarzi...,"Diclofenac-Natrium 3% in Hyaluronsäure 2,5% Gel",Clinical_Drug,3184,3231,C4708783,[T200],Diclofenac diethylammonium 11.6 mg/g cutaneous...,"[ngram, sapbert]",0.062262
1613,08_aktinische-keratose-und-plattenepithelkarzi...,"Diclofenac-Natrium 3% in Hyaluronsäure 2,5% Gel",Clinical_Drug,3184,3231,C2348218,[T200],Sodium Hyaluronate Topical Hydrogel,"[ngram, sapbert]",0.062185


## Lexical Ambiguity

In [46]:
linker.candidate_generator.predict_no_context(['M. Paget'])

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

[{'id': '0_0',
  'offsets': [[0, 8]],
  'text': ['M. Paget'],
  'type': None,
  'normalized': [{'db_id': 'C0029401',
    'db_name': 'UMLS',
    'score': 0.886748731136322,
    'predicted_by': ['ngram', 'sapbert']},
   {'db_id': 'C1368019',
    'db_name': 'UMLS',
    'score': 0.8277765512466431,
    'predicted_by': ['ngram', 'sapbert']},
   {'db_id': 'C1956396',
    'db_name': 'UMLS',
    'score': 0.7899335026741028,
    'predicted_by': ['ngram', 'sapbert']},
   {'db_id': 'C0030186',
    'db_name': 'UMLS',
    'score': 0.7865041494369507,
    'predicted_by': ['ngram', 'sapbert']},
   {'db_id': 'C0238330',
    'db_name': 'UMLS',
    'score': 0.786299467086792,
    'predicted_by': ['ngram', 'sapbert']},
   {'db_id': 'C0410458',
    'db_name': 'UMLS',
    'score': 0.7737199664115906,
    'predicted_by': ['ngram', 'sapbert']},
   {'db_id': 'C0030185',
    'db_name': 'UMLS',
    'score': 0.7730861306190491,
    'predicted_by': ['ngram', 'sapbert']},
   {'db_id': 'C0410457',
    'db_name': 'U